
# Arabic RAG Chatbot Project

#### Build a simple Arabic QA chatbot using Retrieval-Augmented Generation (RAG)


## Load the Dataset
Using the ARCD dataset from Hugging Face — it has about 1.4k Arabic QA pairs


In [2]:
from datasets import load_dataset
import pandas as pd

# Load data
dataset = load_dataset("hsseinmz/arcd")
df_train = pd.DataFrame(dataset["train"])
df_val = pd.DataFrame(dataset["validation"])


In [3]:
# Show sample
df_train.head()

,id,title,context,question,answers
0,969331847966,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- من هو جمال أحمد حمزة خاشقجي؟,"{'text': ['صحفي وإعلامي'], 'answer_start': [73]}"
1,115150665555,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال,{'text': ['حمزة خاشقجي (13 أكتوبر 1958، المدين...
2,74212080718,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- في أي مدينة ولد جمال أحمد حمزة خاشقجي؟ ال,"{'text': ['المدينة المنورة'], 'answer_start': ..."
3,465699296586,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- في أي صحيفة قام بكتابة عمود منذ عام 2017؟ ال,"{'text': ['واشنطن بوست'], 'answer_start': [224]}"
4,564177542570,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- كيف وصفها في الصحف ووسائل الإعلام الدولية؟ ال,{'text': ['وُصف في الصحف وأجهزة الاعلام العالم...


In [4]:
print(df_train.iloc[0])


id                                               969331847966
title                                             جمال خاشقجي
context     جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...
question                       - من هو جمال أحمد حمزة خاشقجي؟
answers      {'text': ['صحفي وإعلامي'], 'answer_start': [73]}
Name: 0, dtype: object


In [5]:
# Just checking sizes and missing values
print(f"Training Data Size: {len(df_train)}")
print(f"Validation Data Size: {len(df_val)}")
print("\nMissing values in training data:")
print(df_train.isnull().sum())

Training Data Size: 693
Validation Data Size: 702

Missing values in training data:
id          0
title       0
context     0
question    0
answers     0
dtype: int64


## Text Cleaning & Normalization
Arabic text can be messy sometimes (extra marks, diacritics, etc.), so here I just clean and normalize it by removing diacritics, unwanted characters, and extra spaces to keep everything consistent.


In [ ]:
import re

def normalize_arabic(text):
    # Normalize Arabic text
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    text = re.sub(r'[^\u0600-\u06FF0-9\s؟.,:؛!]', '', text)
    return re.sub(r'\s+', ' ', text).strip()


df_train_raw = df_train.copy()
# clean columns
for df in [df_train, df_val]:
    for col in ['context', 'question']:
        df[col] = df[col].astype(str).apply(normalize_arabic)
    df['answer_text'] = df['answers'].apply(lambda x: x['text'][0] if x['text'] else "")
    df['answer_text'] = df['answer_text'].astype(str).apply(normalize_arabic)


In [7]:
# Just showing before and after cleaning
sample_text = df_train_raw.loc[20, "context"]
print(" Before cleaning:\n", sample_text)
print("\n After cleaning:\n", normalize_arabic(sample_text))

 Before cleaning:
 مِصرَ أو (رسمياً: جُمهورِيّةُ مِصرَ العَرَبيّةِ) هي دولة عربية تقع في الركن الشمالي الشرقي من قارة أفريقيا، ولديها امتداد آسيوي، حيث تقع شبه جزيرة سيناء داخل قارة آسيا فهي دولة عابرة للقارات، قُدّر عدد سكانها بـ104 مليون نسمة، ليكون ترتيبها الثالثة عشر بين دول العالم بعدد السكان والأكثر سكانا عربيا.

 After cleaning:
 مصر أو رسميا: جمهورية مصر العربية هي دولة عربية تقع في الركن الشمالي الشرقي من قارة أفريقيا، ولديها امتداد آسيوي، حيث تقع شبه جزيرة سيناء داخل قارة آسيا فهي دولة عابرة للقارات، قدر عدد سكانها بـ104 مليون نسمة، ليكون ترتيبها الثالثة عشر بين دول العالم بعدد السكان والأكثر سكانا عربيا.


## Generate Embeddings
Generated embeddings for all the contexts using an Arabic transformer model


In [8]:
from sentence_transformers import SentenceTransformer

# Load Arabic embeddings model
embeddings_model = SentenceTransformer('Omartificial-Intelligence-Space/Arabic-Triplet-Matryoshka-V2')

def generate_embeddings(texts):
    """encode texts and normalize thems """
    return embeddings_model.encode(
        texts,
        batch_size=16,
        normalize_embeddings=True,
        convert_to_tensor=True
    )

train_context_emb = generate_embeddings(df_train["context"].tolist())
print(f"Training Contexts Embeddings Shape: {train_context_emb.shape}")


c:\Users\areen\anaconda3\envs\ml_env\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)



Training Contexts Embeddings Shape: torch.Size([693, 768])


## Qdrant Setup (Vector Database)
Here I connected to Qdrant, a vector database, to store all the context embeddings for easy retrieval.


In [9]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

# connect to Qdrant
client = QdrantClient("http://localhost:6333")


collection_name = "arabic_qa"

# recreate the collection
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=train_context_emb.shape[1],
        distance=Distance.COSINE
    )
)

# upload data points
points = [
    PointStruct(
        id=i,
        vector=train_context_emb[i].cpu().numpy().tolist(),
        payload={
            "context": df_train.iloc[i]["context"],
            "question": df_train.iloc[i]["question"],
            "answer_text": df_train.iloc[i]["answer_text"],
            "id": df_train.iloc[i]["id"],
            "title": df_train.iloc[i]["title"]
        }
    )
    for i in range(len(df_train))
]

operation_info = client.upsert(
    collection_name=collection_name,
    wait=True,
    points=points
)


C:\Users\areen\AppData\Local\Temp\ipykernel_18964\2006596089.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [10]:
print("Upsert complete:", operation_info.status)
print("Total vectors stored:", len(points))

Upsert complete: completed
Total vectors stored: 693


##  Retrieval Function
This part finds the most similar contexts from Qdrant based on the question the user asks.

In [11]:

def retrieve_similar_context(query, top_k=5):
    """Retrieve similar contexts and their scores from the Qdrant collection."""

    query_embedding = embeddings_model.encode(query,normalize_embeddings=True,)

    search_results = client.query_points(
        collection_name=collection_name,
        query=query_embedding,
        limit=top_k
    )

    results = []
    for result in search_results.points:
        results.append({
            "context": result.payload["context"],
            "answer": result.payload.get("answer_text", ""),
            "score": result.score
        })
    return results


In [12]:
# quick test
user_question = "في أي علم ولد ألبرت أينشتاين؟"
similar_contexts = retrieve_similar_context(user_question)

print(" Top-k Retrieved Contexts:")
for i, context in enumerate(similar_contexts, start=1):
    print(f"{i}. [Score: {context['score']:.4f}]")
    print(f"   Context: {context['context']}")
    print(f"   Answer: {context['answer']}\n")


 Top-k Retrieved Contexts:
1. [Score: 0.8055]
   Context: ألبرت أينشتاين بالألمانية: 14 مارس 1879 18 أبريل 1955 عالم فيزياء ألماني المولد، حيث تخلى عن الجنسية الألمانية لاحقا سويسري وأمريكي الجنسية، من أبوين يهوديين، وهو يشتهر بأب النسبية كونه واضع النسبية الخاصة والنسبية العامة الشهيرتين اللتين كانتا اللبنة الأولى للفيزياء النظرية الحديثة، ولقد حاز في عام 1921 على جائزة نوبل في الفيزياء عن ورقة بحثية عن التأثير الكهروضوئي، ضمن ثلاثمائة ورقة علمية أخرى له في تكافؤ المادة والطاقة وميكانيكا الكم وغيرها، وأدت استنتاجاته المبرهنة إلى تفسير العديد من الظواهر العلمية التي فشلت الفيزياء الكلاسيكية في إثباتها.
   Answer: 1879

2. [Score: 0.8055]
   Context: ألبرت أينشتاين بالألمانية: 14 مارس 1879 18 أبريل 1955 عالم فيزياء ألماني المولد، حيث تخلى عن الجنسية الألمانية لاحقا سويسري وأمريكي الجنسية، من أبوين يهوديين، وهو يشتهر بأب النسبية كونه واضع النسبية الخاصة والنسبية العامة الشهيرتين اللتين كانتا اللبنة الأولى للفيزياء النظرية الحديثة، ولقد حاز في عام 1921 على جائزة نوبل في الفيزياء عن ورقة ب

## RAG Generation (Using Arabic GPT-2)
Used an Arabic GPT-2 model to generate answers , basically combining the question with the retrieved context to make a full response.

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load Arabic GPT-2
gpt2_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-medium")
gpt2_model = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-medium")


In [14]:
def generate_prompt(question, retrieved_contexts):
    prompt = "أجب على السؤال التالي بناءً على المعلومات المتاحة أدناه:\n\n"

    for i, ctx in enumerate(retrieved_contexts, 1):
        prompt += f"المعلومة {i}: {ctx['context']}\n"

    prompt += f"\nالسؤال: {question}\nالإجابة :"
    return prompt


In [15]:
# Simple function that cuts the answer
def truncate_answer(answer, stop_chars=['.', '،']):
    for char in stop_chars:
        if char in answer:
            return answer.split(char)[0] + char
    return answer

In [16]:
def generate_answer(question, top_k=1, max_new_tokens=30):

    retrieved_contexts = retrieve_similar_context(question, top_k=top_k)
    prompt_text = generate_prompt(question, retrieved_contexts)
    inputs = gpt2_tokenizer(prompt_text, return_tensors="pt", truncation=True, max_length=512)

    # generate the text
    output = gpt2_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        no_repeat_ngram_size=3,
        do_sample=False ,
        top_p=0.8,
        num_beams=8,
        temperature=0.7,
        pad_token_id=gpt2_tokenizer.eos_token_id,
        eos_token_id=gpt2_tokenizer.eos_token_id

    )

    # decode answer
    answer = gpt2_tokenizer.decode(output[0], skip_special_tokens=True)
    final_answer = answer.replace(prompt_text, "").strip()
    final_answer = truncate_answer(final_answer)


    return final_answer


In [18]:
# test it
question = "في أي علم ولد ألبرت أينشتاين؟"
answer = generate_answer(question)
print("السؤال:", question)
print("الإجابة:", answer)

السؤال: في أي علم ولد ألبرت أينشتاين؟
الإجابة: ولد ألبرت إينشتاين في عام 1879 م في مدينة لايبزغ في ألمانيا ،


In [19]:
# test it

question = "ما عاصمة فلسطين "
answer = generate_answer(question)
print("السؤال:", question)
print("الإجابة:", answer)


السؤال: ما عاصمة فلسطين 
الإجابة: هي مدينة القدس عاصمة دولة فلسطين ،


## Evaluation
calculated BLEU , F1 and Semantic Similarity just to check how good the model’s answers are.

In [20]:
import re
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import util

predictions = []
references = []

df_subset = df_val.head(50)

# Generating and normalizing answers
for i, row in df_subset.iterrows():
    question = row['question']
    true_answer = row['answer_text']

    generated_answer = generate_answer(question)

    norm_pred = normalize_arabic(generated_answer)
    norm_true = normalize_arabic(true_answer)

    predictions.append(norm_pred)
    references.append(norm_true)


In [21]:
smooth_fn = SmoothingFunction().method1

# BLEU-1
bleu_scores = [
    sentence_bleu([ref.split()], pred.split(), smoothing_function=smooth_fn)
    for ref, pred in zip(references, predictions)
]
avg_bleu = np.mean(bleu_scores)

# BLEU-2
bleu_2_scores = [
    sentence_bleu([ref.split()], pred.split(), smoothing_function=smooth_fn, weights=(0.5, 0.5))
    for ref, pred in zip(references, predictions)
]
avg_bleu_2 = np.mean(bleu_2_scores)

# F1 Score
def f1_score(pred, true):
    pred_tokens = set(pred.split())
    true_tokens = set(true.split())
    common = pred_tokens & true_tokens

    if len(common) == 0:
        return 0.0

    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(true_tokens)

    return 2 * (precision * recall) / (precision + recall)

f1_scores = [f1_score(p, r) for p, r in zip(predictions, references)]
avg_f1 = np.mean(f1_scores)


# Semantic Similarity
emb_preds = embeddings_model.encode(predictions, normalize_embeddings=True, convert_to_tensor=True)
emb_refs = embeddings_model.encode(references, normalize_embeddings=True, convert_to_tensor=True)

similarities = util.cos_sim(emb_preds, emb_refs).diagonal().cpu().numpy()
avg_sim = np.mean(similarities)



In [22]:
print(" Evaluation Results:")
print(f"Average BLEU score: {avg_bleu:.4f}")
print(f"Average BLEU-2 score: {avg_bleu_2:.4f}")
print(f"Average F1 score: {avg_f1:.4f}")
print(f"Average Semantic Similarity: {avg_sim:.4f}")


 Evaluation Results:
Average BLEU score: 0.0091
Average BLEU-2 score: 0.0201
Average F1 score: 0.0814
Average Semantic Similarity: 0.3093


The scores are low, but it's expected since we didn’t fine-tune the model on this dataset yet. 
If we fine-tune it, the performance could get way better.

### gird search
 Grid Search for Best Generation Parameters


In [ ]:
# #  Grid Search for Best Generation Parameters
# from itertools import product

# # Define parameter grid
# param_grid = {
#     "top_k": [1, 3, 5],
#     "max_new_tokens": [30, 50, 70],
#     "temperature": [0.7, 0.8, 1.0],
#     "top_p": [0.8, 0.9, 1.0]
# }

# # Generate all combinations
# param_combinations = list(product(
#     param_grid["top_k"],
#     param_grid["max_new_tokens"],
#     param_grid["temperature"],
#     param_grid["top_p"]
# ))

# best_params = None
# best_score = -1

# subset_df = df_val.head(20)

# for top_k, max_new_tokens, temperature, top_p in param_combinations:
#     predictions = []
#     references = []
#     similarities = []

#     for _, row in subset_df.iterrows():
#         question = row['question']
#         true_answer = row['answer_text']

#         generated_answer = generate_answer(
#             question,
#             top_k=top_k,
#             max_new_tokens=max_new_tokens
#         )

#         norm_pred = normalize_arabic(generated_answer)
#         norm_true = normalize_arabic(true_answer)

#         predictions.append(norm_pred)
#         references.append(norm_true)

#         emb_pred = similarity_model.encode(norm_pred, normalize_embeddings=True)
#         emb_ref = similarity_model.encode(norm_true, normalize_embeddings=True)
#         similarities.append(util.cos_sim(emb_pred, emb_ref).item())

#     # Evaluate: BLEU + F1 + Semantic Similarity (average as combined score)
#     smooth_fn = SmoothingFunction().method1
#     bleu_scores = [
#         sentence_bleu([r.split()], p.split(), smoothing_function=smooth_fn)
#         for r, p in zip(references, predictions)
#     ]
#     avg_bleu = np.mean(bleu_scores)

#     f1_scores = [f1_score(p, r) for p, r in zip(predictions, references)]
#     avg_f1 = np.mean(f1_scores)

#     avg_sim = np.mean(similarities)

#     combined_score = (avg_bleu + avg_f1 + avg_sim) / 3

#     if combined_score > best_score:
#         best_score = combined_score
#         best_params = {
#             "top_k": top_k,
#             "max_new_tokens": max_new_tokens,
#             "temperature": temperature,
#             "top_p": top_p
#         }

# print("Best Parameters:", best_params)
# print("Best Combined Score:", best_score)


Best Parameters: {'top_k': 3, 'max_new_tokens': 30, 'temperature': 0.7, 'top_p': 0.8}
Best Combined Score: 0.14345654434674376


# Chatbot interface

In [23]:
import gradio as gr

def chat_bot_interface(user_question):
    top_k = 1
    max_tokens = 30

    answer = generate_answer(user_question, top_k=top_k, max_new_tokens=max_tokens)
    retrieved = retrieve_similar_context(user_question, top_k=top_k)

    context_texts = "\n\n".join([f"{i+1}. {ctx['context']}" for i, ctx in enumerate(retrieved)])
    return answer, context_texts


with gr.Blocks() as demo:
    gr.Markdown("## English RAG Chatbot")
    gr.Markdown("Ask any question in Arabic and let the bot answer ")

    with gr.Row():
        question_input = gr.Textbox(label="Your Question", placeholder="Type your question in Arabic...", lines=2)

    with gr.Row():
        answer_output = gr.Textbox(label="Bot Answer", placeholder="The answer will appear here...", lines=5)
        context_output = gr.Textbox(label="Used Contexts", placeholder="The retrieved contexts will appear here...", lines=10)

    submit_btn = gr.Button("Ask the Bot!")

    submit_btn.click(
        fn=chat_bot_interface,
        inputs=[question_input],
        outputs=[answer_output, context_output]
    )

# Launch the Gradio interface
demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
